In [2]:
import os
import pandas as pd

# Set your local project directory
project_dir = '/home/santiago/Desktop/azheimer/'
os.chdir(project_dir)

# Load the CSV file
csv_path = os.path.join(project_dir, 'DataBaseSubjects.csv')
subjects_df = pd.read_csv(csv_path)

# Verify the contents of the directory (optional)
print(os.listdir(project_dir))

# Verify the contents of the directory
!ls

['VAE_pearson.ipynb', 'ROISignals', 'tensor_processing.log', '3D_matrix', 'new.ipynb', 'DataBaseSubjects.csv', 'gpt.ipynb']
3D_matrix	      gpt.ipynb  ROISignals		VAE_pearson.ipynb
DataBaseSubjects.csv  new.ipynb  tensor_processing.log


In [3]:
import os
import torch
import numpy as np
import scipy.io as sio
from scipy.stats import zscore
from sklearn.metrics import mutual_info_score
from statsmodels.tsa.stattools import grangercausalitytests
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm  # Progress bar
import logging

# Setup logging for error handling
logging.basicConfig(filename='tensor_processing.log', level=logging.ERROR)

# Function to discretize continuous signals into a fixed number of bins
def discretize_signals(signals, num_bins=10):
    discretized_signals = np.zeros_like(signals)
    for i in range(signals.shape[1]):  # Loop over each ROI (column)
        discretized_signals[:, i] = np.digitize(signals[:, i], bins=np.histogram_bin_edges(signals[:, i], bins=num_bins))
    return discretized_signals

# Function to compute mutual information matrix for discretized signals
def compute_mutual_information_matrix(signals, num_bins=10):
    discretized_signals = discretize_signals(signals, num_bins)
    num_rois = discretized_signals.shape[1]
    mi_matrix = np.zeros((num_rois, num_rois))
    
    for i in range(num_rois):
        for j in range(num_rois):
            mi_matrix[i, j] = mutual_info_score(discretized_signals[:, i], discretized_signals[:, j])
    return mi_matrix

# Granger causality function with error handling and logging
def granger_for_pair(args):
    signals, i, j, max_lag = args
    if i != j:
        try:
            test_result = grangercausalitytests(signals[:, [i, j]], max_lag, verbose=False)
            p_value = test_result[max_lag][0]['ssr_ftest'][1]
            return i, j, -np.log(p_value) if p_value > 0 else 0
        except Exception as e:
            logging.error(f"Error computing Granger causality for pair {i}, {j}: {e}")
            return i, j, 0
    return i, j, 0

# Function to compute Granger causality matrix in parallel
def compute_granger_causality_matrix_parallel(signals, max_lag=1, num_workers=4):
    num_rois = signals.shape[1]
    gc_matrix = np.zeros((num_rois, num_rois))
    
    # Prepare the arguments for parallel computation
    args = [(signals, i, j, max_lag) for i in range(num_rois) for j in range(num_rois)]
    
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(granger_for_pair, arg) for arg in args]
        for future in as_completed(futures):
            i, j, value = future.result()
            gc_matrix[i, j] = value
    
    return gc_matrix

# Function to compute the Pearson correlation matrix after z-score normalization
def compute_correlation_matrix(signals):
    # Normalize the signals by channels (rows) using z-score
    zscored_signals = zscore(signals, axis=0)
    # Compute the Pearson correlation matrix
    return np.corrcoef(zscored_signals, rowvar=False)  # rowvar=False to correlate columns

# Function to combine the matrices into a 3D tensor
def combine_matrices_to_tensor(corr_matrix, mi_matrix, gc_matrix):
    combined_tensor = np.stack([corr_matrix, mi_matrix, gc_matrix], axis=0)  # Stack along a new axis (3, 116, 116)
    return combined_tensor

# Function to estimate memory usage of tensors
def estimate_memory_usage(num_subjects, matrix_shape=(3, 116, 116), dtype=torch.float32):
    bytes_per_element = torch.finfo(dtype).bits // 8
    total_elements = np.prod(matrix_shape) * num_subjects
    total_bytes = total_elements * bytes_per_element
    total_gb = total_bytes / (1024**3)  # Convert to GB
    return total_gb

# Parallel function to compute all three matrices for a subject and save tensors to disk incrementally
def process_and_save_subject_matrices(subject_id, signals, group, output_dir):
    corr_matrix = compute_correlation_matrix(signals)
    mi_matrix = compute_mutual_information_matrix(signals)
    gc_matrix = compute_granger_causality_matrix_parallel(signals)
    combined_tensor = combine_matrices_to_tensor(corr_matrix, mi_matrix, gc_matrix)
    
    # Save the tensor to a file instead of keeping everything in memory
    file_name = f'{group}_tensor_{subject_id}.pt'
    tensor_path = os.path.join(output_dir, file_name)
    torch.save(combined_tensor, tensor_path)
    
    return tensor_path

# Initialize dictionaries to store file paths for the 3D tensors
tensor_file_paths = {
    'AD': [],
    'CN': [],
    'Other': []
}

# Progress bar for tensor construction
output_dir = os.path.join(project_dir, 'TensorData')  # Directory for saving tensors
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

with tqdm(total=len(subjects_df), desc="Constructing tensors") as pbar:
    for subject_id in subjects_df['SubjectID']:
        file_path = os.path.join(project_dir, 'ROISignals', f"ROISignals_{subject_id}.mat")
        if os.path.exists(file_path):
            mat_data = sio.loadmat(file_path)
            research_group = subjects_df[subjects_df['SubjectID'] == subject_id]['ResearchGroup'].values[0]
            signals = mat_data['ROISignals'][:, :116]  # Extract the signals
            
            try:
                # Process matrices and save tensors incrementally
                tensor_path = process_and_save_subject_matrices(subject_id, signals, research_group, output_dir)

                # Store file paths in the appropriate group
                if research_group == 'AD':
                    tensor_file_paths['AD'].append(tensor_path)
                elif research_group == 'CN':
                    tensor_file_paths['CN'].append(tensor_path)
                else:
                    tensor_file_paths['Other'].append(tensor_path)

            except Exception as e:
                logging.error(f"Error processing subject {subject_id}: {e}")

        # Update progress
        pbar.update(1)

print("All 3D tensors have been computed, saved, and stored.")

# Estimate memory usage for a large batch
total_gb = estimate_memory_usage(len(subjects_df))
print(f"Estimated memory usage for all tensors: {total_gb:.2f} GB")


Constructing tensors:   0%|          | 0/352 [00:00<?, ?it/s]

/home/santiago/.local/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/home/santiago/.local/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/home/santiago/.local/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/home/santiago/.local/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
Constructing tensors:   0%|          | 1/352 [00:31<3:06:04, 31.81s/it]/home/santiago/.local/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/home/santiago/.local/lib/pyt

All 3D tensors have been computed, saved, and stored.
Estimated memory usage for all tensors: 0.05 GB
